In [14]:
import polars

df = polars.read_csv("./data/Trade_DetailedTradeMatrix.csv", encoding='latin-1')

In [15]:
df.describe()

describe,Reporter Country Code,Reporter Country Code (M49),Reporter Countries,Partner Country Code,Partner Country Code (M49),Partner Countries,Item Code,Item Code (CPC),Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
str,f64,str,str,f64,str,str,f64,str,str,f64,str,f64,f64,str,f64,str
"""count""",4.6508259e7,"""46508259""","""46508259""",4.6508259e7,"""46508259""","""46508259""",4.6508259e7,"""46508259""","""46508259""",4.6508259e7,"""46508259""",4.6508259e7,4.6508259e7,"""46508259""",4.6508259e7,"""46508259"""
"""null_count""",0.0,"""0""","""0""",0.0,"""0""","""0""",0.0,"""0""","""0""",0.0,"""0""",0.0,0.0,"""0""",0.0,"""0"""
"""mean""",132.572986,null,null,132.461901,null,null,559.501377,null,null,5765.099633,null,2008.015594,2008.015594,null,3238.978293,null
"""std""",72.451384,null,null,73.075362,null,null,353.022658,null,null,150.115357,null,9.401291,9.401291,null,522433.35325,null
"""min""",1.0,"""'004""","""Afghanistan""",1.0,"""'004""","""Afghanistan""",15.0,"""'0111""","""Abaca, manila …",5607.0,"""Export Quantit…",1986.0,1986.0,"""1000 Head""",0.0,"""A"""
"""max""",273.0,"""'894""","""Zimbabwe""",299.0,"""'894""","""Zimbabwe""",1296.0,"""'F1293""","""hydrogenated o…",5922.0,"""Import Value""",2021.0,2021.0,"""tonnes""",2.5795e9,"""T"""
"""median""",131.0,null,null,126.0,null,null,562.0,null,null,5622.0,null,2009.0,2009.0,null,21.0,null
"""25%""",68.0,null,null,68.0,null,null,246.0,null,null,5610.0,null,2001.0,2001.0,null,2.0,null
"""75%""",202.0,null,null,202.0,null,null,828.0,null,null,5910.0,null,2016.0,2016.0,null,211.0,null
